In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("SparkSQLExample") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/25 18:15:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 56642)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Library/Frameworks/Pyt

In [14]:
from pyspark.sql import Row

# Create HR employee data
hr_data = [
    (1, "Alice", "HR", 70000, "USA"),
    (2, "Bob", "IT", 90000, "USA"),
    (3, "Charlie", "Finance", 80000, "Canada"),
    (4, "David", "HR", 75000, "Canada"),
    (5, "Eve", "IT", 95000, "India"),
    (6, "Frank", "Finance", 78000, "UK"),
    (7, "Grace", "IT", 88000, "India"),
    (8, "Hank", "Finance", 72000, "USA"),
    (9, "Ivy", "HR", 68000, "UK"),
    (10, "Jack", "IT", 96000, "India"),
    (11, "Kelly", "HR", 71000, "USA"),
    (12, "Liam", "Finance", 82000, "Canada"),
    (13, "Mia", "IT", 97000, "UK"),
    (14, "Noah", "HR", 74000, "Canada"),
    (15, "Olivia", "Finance", 78000, "India")
]

# Convert to Spark DataFrame
columns = ["emp_id", "name", "department", "salary", "country"]
hr_df = spark.createDataFrame(hr_data, columns)

# Register as SQL Table
hr_df.createOrReplaceTempView("hr_table")



In [15]:
spark.sql("""
    SELECT *, 
           RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS rank
    FROM hr_table
""").show()


+------+-------+----------+------+-------+----+
|emp_id|   name|department|salary|country|rank|
+------+-------+----------+------+-------+----+
|    12|   Liam|   Finance| 82000| Canada|   1|
|     3|Charlie|   Finance| 80000| Canada|   2|
|     6|  Frank|   Finance| 78000|     UK|   3|
|    15| Olivia|   Finance| 78000|  India|   3|
|     8|   Hank|   Finance| 72000|    USA|   5|
|     4|  David|        HR| 75000| Canada|   1|
|    14|   Noah|        HR| 74000| Canada|   2|
|    11|  Kelly|        HR| 71000|    USA|   3|
|     1|  Alice|        HR| 70000|    USA|   4|
|     9|    Ivy|        HR| 68000|     UK|   5|
|    13|    Mia|        IT| 97000|     UK|   1|
|    10|   Jack|        IT| 96000|  India|   2|
|     5|    Eve|        IT| 95000|  India|   3|
|     2|    Bob|        IT| 90000|    USA|   4|
|     7|  Grace|        IT| 88000|  India|   5|
+------+-------+----------+------+-------+----+



In [16]:
spark.sql("""
    SELECT *, 
           SUM(salary) OVER (PARTITION BY department ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
    FROM hr_table
""").show()


+------+-------+----------+------+-------+-------------+
|emp_id|   name|department|salary|country|running_total|
+------+-------+----------+------+-------+-------------+
|     8|   Hank|   Finance| 72000|    USA|        72000|
|     6|  Frank|   Finance| 78000|     UK|       150000|
|    15| Olivia|   Finance| 78000|  India|       228000|
|     3|Charlie|   Finance| 80000| Canada|       308000|
|    12|   Liam|   Finance| 82000| Canada|       390000|
|     9|    Ivy|        HR| 68000|     UK|        68000|
|     1|  Alice|        HR| 70000|    USA|       138000|
|    11|  Kelly|        HR| 71000|    USA|       209000|
|    14|   Noah|        HR| 74000| Canada|       283000|
|     4|  David|        HR| 75000| Canada|       358000|
|     7|  Grace|        IT| 88000|  India|        88000|
|     2|    Bob|        IT| 90000|    USA|       178000|
|     5|    Eve|        IT| 95000|  India|       273000|
|    10|   Jack|        IT| 96000|  India|       369000|
|    13|    Mia|        IT| 970

In [17]:
spark.sql("""
    SELECT *, 
           DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dense_rank
    FROM hr_table
""").show()


+------+-------+----------+------+-------+----------+
|emp_id|   name|department|salary|country|dense_rank|
+------+-------+----------+------+-------+----------+
|    12|   Liam|   Finance| 82000| Canada|         1|
|     3|Charlie|   Finance| 80000| Canada|         2|
|     6|  Frank|   Finance| 78000|     UK|         3|
|    15| Olivia|   Finance| 78000|  India|         3|
|     8|   Hank|   Finance| 72000|    USA|         4|
|     4|  David|        HR| 75000| Canada|         1|
|    14|   Noah|        HR| 74000| Canada|         2|
|    11|  Kelly|        HR| 71000|    USA|         3|
|     1|  Alice|        HR| 70000|    USA|         4|
|     9|    Ivy|        HR| 68000|     UK|         5|
|    13|    Mia|        IT| 97000|     UK|         1|
|    10|   Jack|        IT| 96000|  India|         2|
|     5|    Eve|        IT| 95000|  India|         3|
|     2|    Bob|        IT| 90000|    USA|         4|
|     7|  Grace|        IT| 88000|  India|         5|
+------+-------+----------+-

In [18]:
spark.sql("""
    SELECT *, 
           MAX(salary) OVER (PARTITION BY department) AS max_salary
    FROM hr_table
""").show()


+------+-------+----------+------+-------+----------+
|emp_id|   name|department|salary|country|max_salary|
+------+-------+----------+------+-------+----------+
|     3|Charlie|   Finance| 80000| Canada|     82000|
|     6|  Frank|   Finance| 78000|     UK|     82000|
|     8|   Hank|   Finance| 72000|    USA|     82000|
|    12|   Liam|   Finance| 82000| Canada|     82000|
|    15| Olivia|   Finance| 78000|  India|     82000|
|     1|  Alice|        HR| 70000|    USA|     75000|
|     4|  David|        HR| 75000| Canada|     75000|
|     9|    Ivy|        HR| 68000|     UK|     75000|
|    11|  Kelly|        HR| 71000|    USA|     75000|
|    14|   Noah|        HR| 74000| Canada|     75000|
|     2|    Bob|        IT| 90000|    USA|     97000|
|     5|    Eve|        IT| 95000|  India|     97000|
|     7|  Grace|        IT| 88000|  India|     97000|
|    10|   Jack|        IT| 96000|  India|     97000|
|    13|    Mia|        IT| 97000|     UK|     97000|
+------+-------+----------+-

25/02/25 18:53:33 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 177093 ms exceeds timeout 120000 ms
25/02/25 18:53:33 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/25 18:53:35 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$